In [29]:
import numpy as np
from loss import MSELoss, BinaryCrossEntropyLoss
from module import Linear, TanH, Sigmoid, Sequentiel
from optimizer import Optim
from utils import gen_arti
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [30]:
x = np.random.randn(100, 3)
y = np.hstack((np.expand_dims(x[:, 0] + 2 * x[:, 1], 1), np.expand_dims(x[:, 0] + x[:, 1] + 5 * x[:, 2], 1)))

module = Linear(3, 2)
loss = MSELoss()

for _ in range(100):
    yhat = module.forward(x)
    delta1 = loss.backward(y, yhat)

    module.backward_update_gradient(delta1)
    module.update_parameters()
    module.zero_grad()

module.forward(np.array([[1, 2, 3]]))

array([[ 5.00885347, 18.01842961]])

In [31]:
m = Sequentiel(Linear(3, 2))
op = Optim(m, MSELoss())

for _ in range(100):
    op.zero_grad()
    op.step(x, y)

m.forward(np.array([[1, 2, 3]]))

array([[ 5.03636265, 17.9907656 ]])

In [32]:
x, y = gen_arti(data_type=1)
y[y == -1] = 0

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.2)

In [36]:
m = Sequentiel(Linear(2, 2), TanH(), Linear(2, 1), Sigmoid())
op = Optim(m, BinaryCrossEntropyLoss())

for _ in range(1000):
    op.zero_grad()
    op.step(x_train, y_train)

In [39]:
yhat = m.forward(x_test)
yhat[yhat >= .5] = 1
yhat[yhat < .5] = 0

In [40]:
accuracy_score(y_test, yhat)

0.985